# Assignment #3: Movie Dictionary Database

In this problem, we aim to load movie data into a dictionary, summarize it by genre, and performing two simple searches.

Please read all the details [here](https://docs.google.com/document/d/1DQ8d6D1Q3p3qOBKxN8B4twShe9zkI7-WpPtFjQ3y1p4/edit?usp=sharing).

# **Important**
- Do not delete or modify the first line in the given code cell.
- You work must be add in the provided area only.
- You must not change the declaration of the provided functions.
- You are allowed to add your own functions.
- If you want to write your own program to test, add new code cell at the very end of the file and add your code there.


In [ ]:
# uncomment this cell to download the movie database file to colab
!wget https://raw.githubusercontent.com/sothornin/file/main/movies_2010_2013.json

In [ ]:
# MOVIE_DICTIONARY (*** DO NOT DELETE this line or add line before this ***)
# Only add your code in the provided area.
# DO NOT delete or modified the given code in main().

# Function 1
def load_data_to_movie_dict(file_name):
    import json
    # use json.loads to turn string into dictionary
    # return the movies info in dictionary form
    mov = {}
    f = open(file_name, 'r')
    for line in f:
        i = json.loads(line)
        mov[i['_id']['$oid']] = { 'title': i['title'], 'year': int(i['year']['$numberInt']) }
        if 'genres' in i.keys(): mov[i['_id']['$oid']]['genres'] = i['genres']
        if 'tomatoes' in i.keys() and 'viewer' in i['tomatoes'] and 'meter' in i['tomatoes']['viewer']:
            mov[i['_id']['$oid']]['rating'] = int(i['tomatoes']['viewer']['meter']['$numberInt'])
    f.close()
    return mov

# Function 2
def yearly_summary_by_genre(year, movies):
    # return the dictionary of genre as the key and summary of length and min max rating as the value
    ysbg = {}
    for info in movies.values():
        if info['year'] == year and 'genres' in info:
            for genre in info['genres']:
                if genre not in ysbg.keys():
                    ysbg[genre] = { 'num': 1 }
                else:
                    ysbg[genre]['num'] += 1

                if 'rating' in info:
                    if 'rating' not in ysbg[genre]:
                        ysbg[genre]['rating'] = {
                            'avg': info['rating'],
                            'min': info['rating'],
                            'max': info['rating']}
                    else:
                        ysbg[genre]['rating']['avg'] += info['rating']
                        if info['rating'] < ysbg[genre]['rating']['min']:
                            ysbg[genre]['rating']['min'] = info['rating']
                        if info['rating'] > ysbg[genre]['rating']['max']:
                            ysbg[genre]['rating']['max'] = info['rating']

    for genre in ysbg: ysbg[genre]['rating']['avg'] = int(round(ysbg[genre]['rating']['avg'] / ysbg[genre]['num']))
    return ysbg

def search_g(sgenre, movies):
    mov_ids = []
    for ids, info in movies.items():
        if 'genres' in info:
            for i in info['genres']:
                if i.lower() == sgenre: mov_ids.append(ids)
    return mov_ids

def search_r(srating, movies):
    mov_ids = [ids for ids, info in movies.items() if 'rating' in info and info['rating'] == srating]
    return mov_ids

def search_y(syear, movies):
    mov_ids = [ids for ids, info in movies.items() if info['year'] == syear]
    return mov_ids

# Function 3
def search_movie_keys(query, movies):
    # return a list of ids
    sgenre, srating, syear = '', 0, 0
    mov_ids = []

    for k in query:
        if k == 'genres': sgenre = query['genres'].lower()
        elif k == 'rating': srating = query['rating']
        elif k == 'year': syear = query['year']
        else: return []

    if sgenre != '': g = set(search_g(sgenre, movies))
    if srating != 0: r = set(search_r(srating, movies))
    if syear != 0: y = set(search_y(syear, movies))

    if sgenre != '' and srating != 0 and syear != 0: mov_ids = list(r.intersection(y, g))
    elif sgenre == '' and srating == 0 and syear != 0: mov_ids = list(y)
    elif sgenre == '' and srating != 0 and syear == 0: mov_ids = list(r)
    elif sgenre != '' and srating == 0 and syear == 0: mov_ids = list(g)
    elif sgenre != '' and srating != 0 and syear == 0: mov_ids = list(g.intersection(r))
    elif sgenre != '' and srating == 0 and syear != 0: mov_ids = list(g.intersection(y))
    elif sgenre == '' and srating != 0 and syear != 0: mov_ids = list(r.intersection(y))

    temp = sorted([[movies[i]['title'], movies[i]['year']] for i in mov_ids])
    mov_ids = [ids for ii in temp for ids, info in movies.items() if info['title'] == ii[0]]
    return mov_ids

def word_deconstruct(query):
    deconstructed_words = [[query]]
    words = query.split()
    word_count = len(words)

    for _ in range(1, len(words) - 1):
        temp = []
        for end in range(len(words), -1, -1):
            for start in range(len(words), -1, -1):
                if end - start == word_count - 1:
                    temp.append(' '.join(words[start:end]))
        word_count -= 1
        deconstructed_words.append(temp)

    deconstructed_words.append(list(set(words)))
    return deconstructed_words

# Function 4
def search_movie_title(query, movies):
    # return a list of ids
    query = query.lower()
    temp, mov_ids, detected = [], [], []

    if len(query.split()) == 1:
        for ids, info in movies.items():
            c = len(set(query.split()).intersection(set([info['title'].lower().split()])))
            if c > 0: temp.append([-c, len(info['title'].split()), info['title'], info['year'], ids])

        mov_ids = [ii[4] for ii in sorted(temp)]

    else:
        for ii in word_deconstruct(query):
            for info in movies.values():
                for jj in word_deconstruct(info['title'].lower()):
                    if len(set(ii).intersection(set(jj))) > 0:
                        detected = ii
                        break
                if detected: break
            if detected: break

        if detected == []: return []

        for ids, info in movies.items():
            for words in word_deconstruct(info['title'].lower()):
                c = len(set(detected).intersection(set(words)))
                if c > 0: temp.append((ids, -c))

        temp1 = [[c, len(movies[ids]['title'].split()), movies[ids]['title'], movies[ids]['year'], ids] for ids, c in list(set(temp))]
        mov_ids = [ii[4] for ii in sorted(temp1)]

    return mov_ids

#------------------------------------------------------------#
# DO NOT DELETE OR MODIFIED THE CODE BELOW
#------------------------------------------------------------#

# This function will help you print out the movies' details from list of movie ids
def show_movie_info(ids, movies):
  for n,i in enumerate(ids):
    if 'title' in movies[i]:
      print('Title: {}'.format(movies[i]['title']))
    if 'year' in movies[i]:
      print('Year: {}'.format(movies[i]['year']))
    if 'genres' in movies[i]:
      print('Genres: {}'.format(', '.join(movies[i]['genres'])))
    if 'rating' in movies[i]:
      print('Rating: {}'.format(movies[i]['rating']))
    if n < len(ids) - 1:
      print('=============================================')

#------------------------------------------------------------#
# *** MAIN PART ****
movies = load_data_to_movie_dict('movies_2010_2013.json')

# TEST
To speed up the testing of your program, some of test cases of some functions are given, but might not cover all possibilies.  You should add more test cases using provided test case as an example.

Uncomment the following cell to test if your function can correctly return the expected output.

Note: The variable `movies` is the output of the function `load_data_to_movie_dict` function

### Test1:

Expected output: `{'title': "Don't Be Afraid of the Dark",
 'year': 2010,
 'genres': ['Fantasy', 'Horror', 'Thriller'],
 'rating': 34}`

In [ ]:
movies['573a13c2f29313caabd68170']

### Test2:

Expected output: `{'title': 'Ugly', 'year': 2013, 'genres': ['Crime', 'Drama', 'Mystery']}`

In [ ]:
movies['573a13e3f29313caabdc006c']

### Test3:

Expected output: `4234`

In [ ]:
len(movies)

### Test4:

Expected output: `{'num': 100, 'rating': {'avg': 42, 'min': 21, 'max': 90}}`

In [ ]:
yearly_summary_by_genre(2013,movies)['Adventure']

### Test5:

Expected output: `{'num': 71, 'rating': {'avg': 29, 'min': 10, 'max': 94}}`

In [ ]:
yearly_summary_by_genre(2010,movies)['Horror']

### Test6:

Expected output: `['573a13c6f29313caabd736c7', '573a13bcf29313caabd56abe']`

In [ ]:
search_movie_keys({'genres':'Drama','rating':89,'year':2010},movies)

### Test7:

Expected output: `[]`

In [ ]:
search_movie_keys({'genre':'Drama','rat':89,'year':2010},movies)

### Test8:

Expected output: `['573a13c4f29313caabd6cc80']`

In [ ]:
search_movie_keys({'genres':'actioN', 'rating': 90, 'year':2012},movies)

### Test9:

Expected output: `['573a13cbf29313caabd80583',
 '573a13d6f29313caabd9f52b',
 '573a13daf29313caabdabca6']`

In [ ]:
movie_ids = search_movie_title('my way',movies)
movie_ids

In [ ]:
show_movie_info(movie_ids, movies)

### Test10:

Expected output: `['573a13bdf29313caabd59b05',
 '573a13cff29313caabd88ec2',
 '573a13c7f29313caabd73d01']`

In [ ]:
movie_ids = search_movie_title('the way and way back',movies)
movie_ids

In [ ]:
show_movie_info(movie_ids, movies)

### Test11:

Expected output: `['573a13d6f29313caabd9fae9', '573a13daf29313caabdacaa5', '573a13d9f29313caabda8a6e', '573a13daf29313caabdab37d']`

In [ ]:
movie_ids = search_movie_title('italy china italy',movies)
movie_ids

In [ ]:
show_movie_info(movie_ids, movies)